### Data Preprocessing

In [1]:
import pandas as pd
import ast
import re
import numpy as np

In [2]:
# First, let's check the file encoding
import pandas as pd

# Try reading with different encodings if needed
try:
    df = pd.read_csv(
        "C:/Users/mahmu/Downloads/DataAnalyticsProjects/product-recommender-ai/data/mobiledokan_products.csv",
        quotechar='"',
        quoting=1,  # QUOTE_MINIMAL
        on_bad_lines='warn',  # This replaces error_bad_lines in newer versions
        encoding='utf-8'
    )
    print("Successfully read the file with UTF-8 encoding")
except UnicodeDecodeError:
    try:
        df = pd.read_csv(
            "C:/Users/mahmu/Downloads/DataAnalyticsProjects/product-recommender-ai/data/mobiledokan_products.csv",
            quotechar='"',
            quoting=1,
            on_bad_lines='warn',
            encoding='latin1'  # Try latin1 if UTF-8 fails
        )
        print("Successfully read the file with latin1 encoding")
    except Exception as e:
        print(f"Error reading file: {e}")

# Display the first few rows if successful
if 'df' in locals():
    display(df.head())

Successfully read the file with UTF-8 encoding


,name,brand,model,price,url,img_url,display_type,screen_size_inches,display_resolution,pixel_density_ppi,...,face_unlock,light_sensor,infrared,fm_radio,operating_system,os_version,user_interface,status,made_by,release_date
0,Samsung Galaxy Z Fold7 (512GB),Samsung,Galaxy Z Fold7 (512GB),"235,000(Unofficial)",https://www.mobiledokan.com/mobile/samsung-gal...,https://www.mobiledokan.com/media/samsung-gala...,Foldable Dynamic LTPO AMOLED 2X,8.0 inches (207.4 cm2),1968x2184 px,368 ppi,...,Yes,"Light sensor, Proximity sensor, Accelerometer,...",NaN,NaN,Android,v16,One UI 8,Available,South Korea,9-Jul-25
1,Apple iPhone 17 Pro Max,Apple,iPhone 17 Pro Max,"190,000(Expected)",https://www.mobiledokan.com/mobile/apple-iphon...,https://www.mobiledokan.com/media/apple-iphone...,LTPO Super Retina XDR OLED,6.9 inches (116.3 cm),1320x2868 px (FHD+),458 ppi,...,Face ID,"Light sensor, Proximity sensor, Accelerometer,...",NaN,NaN,iOS,v26,NaN,Rumored,US,Not announced yet
2,Apple iPhone 17 Air,Apple,iPhone 17 Air,"170,000(Expected)",https://www.mobiledokan.com/mobile/apple-iphon...,https://www.mobiledokan.com/media/apple-iphone...,Super Retina XDR OLED,6.9 inches (17.53 cm),1320x2868 px (FHD+),458 ppi,...,Face ID,"Light sensor, Proximity sensor, Accelerometer,...",NaN,NaN,iOS,v19,NaN,Rumored,US,Not announced yet
3,OnePlus Nord 5,OnePlus,Nord 5,"53,999(Official)",https://www.mobiledokan.com/mobile/oneplus-nord-5,https://www.mobiledokan.com/media/oneplus-nord...,Swift AMOLED,6.83 inches (17.35 cm),1272x2800 px (FHD+),450 ppi,...,Yes,"Light sensor, Proximity sensor, Accelerometer,...",Yes,NaN,Android,v15,OxygenOS 15.0,Available,China,9-Jul-25
4,Xiaomi Redmi K80 Ultra,Xiaomi,Redmi K80 Ultra,"80,000(Expected)",https://www.mobiledokan.com/mobile/xiaomi-redm...,https://www.mobiledokan.com/media/xiaomi-redmi...,OLED,6.83 inches (17.35 cm),1280x2772 px (FHD+),447 ppi,...,Yes,"Light sensor, Proximity sensor, Accelerometer,...",Yes,NaN,Android,v15,HyperOS 2.1,Rumored,China,Not announced yet


### Data Cleaning

In [3]:
# Clean price column by removing "?." prefix
df['price'] = df['price'].str.replace('?.', '', regex=False)

In [4]:
# Create price_original column with numeric values
df['price_original'] = df['price'].str.replace(',', '').str.extract('(\d+)').astype(float)

In [5]:
df[['price','price_original']].head(10)

,price,price_original
0,"235,000(Unofficial)",235000.0
1,"190,000(Expected)",190000.0
2,"170,000(Expected)",170000.0
3,"53,999(Official)",53999.0
4,"80,000(Expected)",80000.0
5,"110,000(Unofficial)",110000.0
6,"12,499(Official)",12499.0
7,"35,000(Expected)",35000.0
8,"8,000(Expected)",8000.0
9,"19,999(Official)",19999.0


##### Price-based Features

In [6]:
# Price categories
df['price_category'] = pd.cut(df['price_original'], 
    bins=[0, 20000, 40000, 60000, 100000, float('inf')],
    labels=['Budget', 'Mid-range', 'Upper Mid-range', 'Premium', 'Flagship'])

df[["name", "price", "price_category"]].head(10)

,name,price,price_category
0,Samsung Galaxy Z Fold7 (512GB),"235,000(Unofficial)",Flagship
1,Apple iPhone 17 Pro Max,"190,000(Expected)",Flagship
2,Apple iPhone 17 Air,"170,000(Expected)",Flagship
3,OnePlus Nord 5,"53,999(Official)",Upper Mid-range
4,Xiaomi Redmi K80 Ultra,"80,000(Expected)",Premium
5,Nothing Phone 3,"110,000(Unofficial)",Flagship
6,Tecno Spark 40C,"12,499(Official)",Budget
7,Realme 15 Pro,"35,000(Expected)",Mid-range
8,Tecno POP 10,"8,000(Expected)",Budget
9,Tecno Spark 40 Pro,"19,999(Official)",Budget


In [7]:
# Function to convert storage to GB
def convert_to_gb(value):
    if pd.isna(value):
        return None
    value = str(value).lower()
    if 'gb' in value:
        return float(value.replace('gb', '').strip())
    elif 'mb' in value:
        return float(value.replace('mb', '').strip()) / 1024
    else:
        return None

# Function to convert RAM to GB
def convert_ram_to_gb(value):
    if pd.isna(value):
        return None
    value = str(value).lower()
    # Extract numbers from the string
    import re
    numbers = re.findall(r'\d+', value)
    if numbers:
        num = float(numbers[0])
        if 'gb' in value:
            return num
        elif 'mb' in value:
            return num / 1024
        else:
            # If no unit specified, assume GB
            return num
    return None

# Convert storage and RAM to GB
df['storage_gb'] = df['internal_storage'].apply(convert_to_gb)
df['ram_gb'] = df['ram'].apply(convert_ram_to_gb)

# Calculate price per GB and round to 2 decimal places
df['price_per_gb'] = (df['price_original'] / df['storage_gb']).round(2)
df['price_per_gb_ram'] = (df['price_original'] / df['ram_gb']).round(2)

# Display results
df[["name", "price", "internal_storage", "ram", 
          "storage_gb", "ram_gb", "price_per_gb", "price_per_gb_ram"]].head(10)

,name,price,internal_storage,ram,storage_gb,ram_gb,price_per_gb,price_per_gb_ram
0,Samsung Galaxy Z Fold7 (512GB),"235,000(Unofficial)",512 GB,12 GB,512.0,12.0,458.98,19583.33
1,Apple iPhone 17 Pro Max,"190,000(Expected)",256 GB,12 GB,256.0,12.0,742.19,15833.33
2,Apple iPhone 17 Air,"170,000(Expected)",128 GB,8 GB,128.0,8.0,1328.12,21250.00
3,OnePlus Nord 5,"53,999(Official)",512 GB,12 GB,512.0,12.0,105.47,4499.92
4,Xiaomi Redmi K80 Ultra,"80,000(Expected)",1 TB,24 GB,NaN,24.0,NaN,3333.33
5,Nothing Phone 3,"110,000(Unofficial)",256 GB,12 GB,256.0,12.0,429.69,9166.67
6,Tecno Spark 40C,"12,499(Official)",128 GB,4 GB,128.0,4.0,97.65,3124.75
7,Realme 15 Pro,"35,000(Expected)",256 GB,12 GB,256.0,12.0,136.72,2916.67
8,Tecno POP 10,"8,000(Expected)",64 GB,3 GB,64.0,3.0,125.00,2666.67
9,Tecno Spark 40 Pro,"19,999(Official)",128 GB,8 GB,128.0,8.0,156.24,2499.88


##### Display-based Features

In [8]:
# Screen size in numeric format
df['screen_size_numeric'] = df['screen_size_inches'].str.extract('(\d+\.?\d*)').astype(float)

# Resolution in numeric format (width x height)
df['resolution_width'] = df['display_resolution'].str.extract('(\d+)x').astype(float)
df['resolution_height'] = df['display_resolution'].str.extract('x(\d+)').astype(float)

# Pixels per inch (PPI) in numeric format
df['ppi_numeric'] = df['pixel_density_ppi'].str.extract('(\d+)').astype(float)

# Refresh rate in numeric format
df['refresh_rate_numeric'] = df['refresh_rate_hz'].str.extract('(\d+)').astype(float)

df[["name", "screen_size_inches", "display_resolution", "pixel_density_ppi", "refresh_rate_hz", 
  'screen_size_numeric', 'resolution_width', 'resolution_height', 'ppi_numeric', 'refresh_rate_numeric']].head()


,name,screen_size_inches,display_resolution,pixel_density_ppi,refresh_rate_hz,screen_size_numeric,resolution_width,resolution_height,ppi_numeric,refresh_rate_numeric
0,Samsung Galaxy Z Fold7 (512GB),8.0 inches (207.4 cm2),1968x2184 px,368 ppi,120 Hz,8.00,1968.0,2184.0,368.0,120.0
1,Apple iPhone 17 Pro Max,6.9 inches (116.3 cm),1320x2868 px (FHD+),458 ppi,120 Hz,6.90,1320.0,2868.0,458.0,120.0
2,Apple iPhone 17 Air,6.9 inches (17.53 cm),1320x2868 px (FHD+),458 ppi,120 Hz,6.90,1320.0,2868.0,458.0,120.0
3,OnePlus Nord 5,6.83 inches (17.35 cm),1272x2800 px (FHD+),450 ppi,144 Hz,6.83,1272.0,2800.0,450.0,144.0
4,Xiaomi Redmi K80 Ultra,6.83 inches (17.35 cm),1280x2772 px (FHD+),447 ppi,144 Hz,6.83,1280.0,2772.0,447.0,144.0


In [11]:
def get_display_score(phone):
    """
    Calculate display score out of 100 based on resolution, PPI, and refresh rate.
    """
    score = 0
    weights = {
        'resolution': 0.4,  # 40% weight for resolution
        'ppi': 0.3,         # 30% weight for pixel density
        'refresh_rate': 0.3 # 30% weight for refresh rate
    }
    
    # Resolution score (0-40 points)
    if not pd.isna(phone['resolution_width']) and not pd.isna(phone['resolution_height']):
        # Calculate total pixels (in millions)
        total_pixels = (phone['resolution_width'] * phone['resolution_height']) / 1000000
        
        # Normalize to 0-100 scale (assuming 4K UHD = 8.3M pixels is max)
        # 4K UHD (3840x2160) = 8.3M pixels
        resolution_score = min(total_pixels / 8.3 * 100, 100)
        score += resolution_score * weights['resolution']
    
    # PPI score (0-30 points)
    if not pd.isna(phone['ppi_numeric']):
        # Normalize to 0-100 scale (assuming 500+ PPI is excellent)
        ppi_score = min(phone['ppi_numeric'] / 500 * 100, 100)
        score += ppi_score * weights['ppi']
    
    # Refresh rate score (0-30 points)
    if not pd.isna(phone['refresh_rate_numeric']):
        # Normalize to 0-100 scale (assuming 120Hz is excellent)
        refresh_score = min(phone['refresh_rate_numeric'] / 120 * 100, 100)
        score += refresh_score * weights['refresh_rate']
    
    # Adjust for missing features
    available_features = sum(1 for x in [
        not pd.isna(phone['resolution_width']) and not pd.isna(phone['resolution_height']),
        not pd.isna(phone['ppi_numeric']),
        not pd.isna(phone['refresh_rate_numeric'])
    ] if x)
    
    if available_features == 0:
        return 0
    
    # Scale the score based on available features
    final_score = score / sum(weights[k] for k, v in weights.items() 
                             if k == 'resolution' and not pd.isna(phone['resolution_width']) or
                                k == 'ppi' and not pd.isna(phone['ppi_numeric']) or
                                k == 'refresh_rate' and not pd.isna(phone['refresh_rate_numeric']))
    
    return round(final_score, 2)

# Apply the enhanced display score function
df['display_score'] = df.apply(get_display_score, axis=1)

# Display results
df[["name", "display_score"]].head()

,name,display_score
0,Samsung Galaxy Z Fold7 (512GB),72.79
1,Apple iPhone 17 Pro Max,75.72
2,Apple iPhone 17 Air,75.72
3,OnePlus Nord 5,74.16
4,Xiaomi Redmi K80 Ultra,73.92


##### Camera-based Features

In [12]:
# Function to convert camera setup to count
def get_camera_count(value, main_camera=None):
    """
    Calculate camera count from camera_setup or main_camera column.
    """
    # First try to get count from camera_setup
    if not pd.isna(value):
        value = str(value).lower()
        if value == 'single':
            return 1
        elif value == 'dual':
            return 2
        elif value == 'triple':
            return 3
        elif value == 'quad':
            return 4
        elif value == 'penta':
            return 5
        elif 'cameras' in value:
            # Try to extract number from strings like "5 Cameras"
            match = re.search(r'(\d+)', value)
            if match:
                return int(match.group(1))
    
    # If camera_setup didn't work, try main_camera
    if not pd.isna(main_camera):
        main_camera_str = str(main_camera)
        if '+' in main_camera_str:
            # Count the number of cameras by counting '+'
            return main_camera_str.count('+') + 1
        elif 'MP' in main_camera_str or re.search(r'\d+', main_camera_str):
            # If it has MP or any number, it's at least one camera
            return 1
    
    return 0

# Apply the enhanced function
df['camera_count'] = df.apply(lambda row: get_camera_count(row['camera_setup'], row['main_camera']), axis=1)

# Display results
print(df[["name", "camera_setup", "camera_count"]].head())

                             name camera_setup  camera_count
0  Samsung Galaxy Z Fold7 (512GB)       Triple             3
1         Apple iPhone 17 Pro Max       Triple             3
2             Apple iPhone 17 Air       Single             1
3                  OnePlus Nord 5         Dual             2
4          Xiaomi Redmi K80 Ultra       Triple             3


In [13]:

# Function to extract camera MP values
def extract_camera_mp(value):
    """
    Extract megapixel value from camera specification string.
    Works with both main_camera and front_camera columns.
    """
    if pd.isna(value):
        return None
    
    value_str = str(value).strip()
    if not value_str:
        return None

    # For formats like "48+8+2MP" or "48MP"
    if '+' in value_str:
        # Extract the first (main) camera MP value
        first_camera = value_str.split('+')[0]
        mp_match = re.search(r'(\d+\.?\d*)', first_camera)
        if mp_match:
            return float(mp_match.group(1))
    else:
        # For single camera format like "48MP"
        mp_match = re.search(r'(\d+\.?\d*)\s*MP', value_str, re.IGNORECASE)
        if mp_match:
            return float(mp_match.group(1))
        
        # Try without the MP suffix (some entries might just have the number)
        mp_match = re.search(r'(\d+\.?\d*)', value_str)
        if mp_match:
            return float(mp_match.group(1))

    return None

# Apply the function to create camera MP columns
# First try the new columns, then fall back to the old ones
df['primary_camera_mp'] = df.apply(
    lambda row: extract_camera_mp(row['main_camera']) if not pd.isna(row['main_camera']) 
    else extract_camera_mp(row['primary_camera_resolution']), 
    axis=1
)

df['selfie_camera_mp'] = df.apply(
    lambda row: extract_camera_mp(row['front_camera']) if not pd.isna(row['front_camera']) 
    else extract_camera_mp(row['selfie_camera_resolution']), 
    axis=1
)

# Display results (showing original and new column)
print(df[["name", "primary_camera_mp", "selfie_camera_mp"]].tail(15))

                              name  primary_camera_mp  selfie_camera_mp
5154           Apple iPhone 17 Pro               48.0              12.0
5155  Xiaomi Redmi Note 14 Pro Max               50.0              24.0
5156            Tecno Camon 50 Pro               50.0              50.0
5157                Realme Note 70               32.0               5.0
5158                 Redmi K90 Pro               50.0              24.0
5159             Motorola Razr 50s               50.0              32.0
5160                    Vivo Y300i               50.0               5.0
5161           Xiaomi Redmi 15 Pro               50.0              20.0
5162                  Vivo iQOO 14               50.0              32.0
5163                Huawei Mate XT               50.0               8.0
5164        Samsung Galaxy Z Flip7               50.0              10.0
5165                     Vivo Y18e               13.0               5.0
5166    Infinix Hot 50 Pro Plus 5G               50.0           

In [15]:
# Enhanced camera score calculation
def get_camera_score(phone):
    """
    Calculate camera score out of 100 based on camera count, primary camera MP, and selfie camera MP.
    Uses main_camera and front_camera columns when available.
    """
    score = 0
    weights = {
        'camera_count': 20,      # 20 points for number of cameras
        'primary_camera_mp': 50,  # 50 points for primary camera resolution
        'selfie_camera_mp': 30    # 30 points for selfie camera resolution
    }
    
    # Camera Count Score (0-20 points)
    if not pd.isna(phone['camera_count']) and phone['camera_count'] > 0:
        # Scale camera count score to 20 points
        # Assuming 4+ cameras is considered high-end
        camera_count_score = min(phone['camera_count'], 4) / 4 * 100
        score += camera_count_score * weights['camera_count'] / 100

    # Primary Camera Score (0-50 points)
    if not pd.isna(phone['primary_camera_mp']):
        # Scale primary camera MP score to 50 points
        # Assuming 200MP is the highest resolution
        primary_camera_score = min(phone['primary_camera_mp'] / 200 * 100, 100)
        score += primary_camera_score * weights['primary_camera_mp'] / 100

    # Selfie Camera Score (0-30 points)
    if not pd.isna(phone['selfie_camera_mp']):
        # Scale selfie camera MP score to 30 points
        # Assuming 64MP is the highest resolution
        selfie_camera_score = min(phone['selfie_camera_mp'] / 64 * 100, 100)
        score += selfie_camera_score * weights['selfie_camera_mp'] / 100

    return round(score, 2)  # Round to 2 decimal places

# Apply the enhanced camera score function
df['camera_score'] = df.apply(get_camera_score, axis=1)

# Display results
print("\nSample of phones with their camera scores:")
df[["name", "camera_setup", "main_camera", "front_camera", 
          "camera_count", "primary_camera_mp", "selfie_camera_mp", "camera_score"]].head()


Sample of phones with their camera scores:


,name,camera_setup,main_camera,front_camera,camera_count,primary_camera_mp,selfie_camera_mp,camera_score
0,Samsung Galaxy Z Fold7 (512GB),Triple,200+10+12MP,10MP,3,200.0,10.0,69.69
1,Apple iPhone 17 Pro Max,Triple,48+48+48MP,24MP,3,48.0,24.0,38.25
2,Apple iPhone 17 Air,Single,48MP,12MP,1,48.0,12.0,22.62
3,OnePlus Nord 5,Dual,50+8MP,50MP,2,50.0,50.0,45.94
4,Xiaomi Redmi K80 Ultra,Triple,50+8+2MP,20MP,3,50.0,20.0,36.88


##### Battery-based Features

In [16]:
# Battery capacity in numeric format
df['battery_capacity_numeric'] = df['capacity'].str.extract('(\d+)').astype(float)

# Has fast charging (boolean)
df['has_fast_charging'] = df['quick_charging'].notna()

# Has wireless charging (boolean)
df['has_wireless_charging'] = df['wireless_charging'].notna()

In [17]:
# Function to extract wattage from quick_charging string
def extract_wattage(value):
    if pd.isna(value):
        return None
    
    value_str = str(value).strip()
    if not value_str:
        return None
    
    # Use regex to find a number (integer or float) followed by 'W'
    wattage_match = re.search(r'(\d+\.?\d*)\s*W', value_str, re.IGNORECASE)
    if wattage_match:
        return float(wattage_match.group(1))
    
    return None

# Apply the function to create the 'charging_wattage' column
df['charging_wattage'] = df['quick_charging'].apply(extract_wattage)


In [18]:
# Calculate the 99th percentile for battery capacity, ignoring NaNs
battery_capacity_99_percentile = df['battery_capacity_numeric'].quantile(0.99)

# Calculate the max wattage
max_wattage = df['charging_wattage'].max()

def get_battery_score_percentile(phone, capacity_percentile, max_wattage_val):
    """
    Calculate battery score out of 100 based on capacity, charging speed, and wireless charging.
    """
    score = 0
    weights = {
        'capacity': 50,  # 50 points for battery capacity
        'charging_wattage': 40,  # 40 points for charging speed
        'wireless_charging': 10  # 10 points for wireless charging
    }

    # Battery Capacity Score (0-50 points)
    if not pd.isna(phone['battery_capacity_numeric']) and capacity_percentile > 0:
        # Scale the capacity score to 50 points
        scaled_capacity = min(phone['battery_capacity_numeric'], capacity_percentile) / capacity_percentile
        score += scaled_capacity * weights['capacity']

    # Charging Speed Score (0-40 points)
    if not pd.isna(phone['charging_wattage']) and max_wattage_val > 0:
        # Scale the charging speed score to 40 points
        scaled_wattage = min(phone['charging_wattage'], max_wattage_val) / max_wattage_val
        score += scaled_wattage * weights['charging_wattage']

    # Wireless Charging Score (0-10 points)
    if 'has_wireless_charging' in phone and not pd.isna(phone['has_wireless_charging']) and phone['has_wireless_charging']:
        score += weights['wireless_charging']

    return round(score, 2)  # Round to 2 decimal places

# Apply the function to calculate battery scores
df['battery_score'] = df.apply(
    lambda row: get_battery_score_percentile(row, battery_capacity_99_percentile, max_wattage),
    axis=1
)

# Display results
print("\nSample of phones with their battery scores:")
df[["name", "capacity", "quick_charging", "wireless_charging",
          "battery_capacity_numeric", "charging_wattage", "has_wireless_charging", "battery_score"]].head()


Sample of phones with their battery scores:


,name,capacity,quick_charging,wireless_charging,battery_capacity_numeric,charging_wattage,has_wireless_charging,battery_score
0,Samsung Galaxy Z Fold7 (512GB),4400 mAh,"25W wired, QC2.0, 50% in 30 min",15W wireless,4400.0,25.0,True,44.72
1,Apple iPhone 17 Pro Max,4700 mAh,"20 Wired, PD2.0, 50% in 30 min (advertised)","25W wireless (MagSafe), 15W wireless (China only)",4700.0,20.0,True,45.97
2,Apple iPhone 17 Air,3000 mAh,"Fast, 20W: 50 % in 30 minutes","25W wireless (MagSafe), 15W wireless (China only)",3000.0,20.0,True,34.17
3,OnePlus Nord 5,6800 mAh,"80W wired, 33W PPS, 18W PD, 100% in just 54 min",NaN,6800.0,80.0,False,60.56
4,Xiaomi Redmi K80 Ultra,7410 mAh,"100W wired, PD3.0",NaN,7410.0,100.0,False,66.67


In [20]:
# Display phones with highest battery scores
print("\nPhones with highest battery scores:")
df.nlargest(10, 'battery_score')[["name", "capacity", "battery_capacity_numeric", "charging_wattage", "battery_score"]]


Phones with highest battery scores:


,name,capacity,battery_capacity_numeric,charging_wattage,battery_score
624,Ulefone Armor 28 Ultra,10600 mAh,10600.0,120.0,80.00
57,Realme GT7 Dream Edition,7000 mAh,7000.0,120.0,78.61
90,Realme GT 7,7000 mAh,7000.0,120.0,78.61
1633,Vivo iQOO 11 Pro,4700 mAh,4700.0,200.0,75.97
69,Honor 400 Pro (China),7200 mAh,7200.0,90.0,75.00
1332,Realme GT5 240W,4600 mAh,4600.0,240.0,71.94
1590,Realme GT Neo 5 240W,4600 mAh,4600.0,240.0,71.94
506,Xiaomi Redmi K80 Pro,6000 mAh,6000.0,120.0,71.67
5158,Redmi K90 Pro,6000 mAh,6000.0,120.0,71.67
82,ZTE nubia Red Magic 10S Pro Plus,7500 mAh,7500.0,120.0,70.00


##### Performance-based Features

In [21]:
import pandas as pd
import re
import numpy as np

# --- Step 1: Load Processor Rankings from CSV ---
processor_rankings = {}
try:
    # Read the CSV file
    processor_df = pd.read_csv('processor_ratings.csv')
    
    # Create a dictionary mapping processor information
    processor_rankings = {}
    for _, row in processor_df.iterrows():
        # For processors with "Other" company, only use processor name
        if row['Company'].lower() == 'other':
            key = row['Processor'].lower().strip()
        else:
            # For other companies, use company + processor
            key = f"{row['Company']} {row['Processor']}".lower().strip()
            
        # Store rank, GPU, and company info
        processor_rankings[key] = {
            'rank': row['Rank'],
            'gpu': row['GPU'].lower().strip() if pd.notna(row['GPU']) else None,
            'company': row['Company'].lower().strip(),
            'processor_name': row['Processor'].lower().strip() 
        }
    
    print(f"Successfully loaded {len(processor_rankings)} processor rankings from processor_ratings.csv")
except FileNotFoundError:
    print("Error: processor_ratings.csv not found. Processor rankings will not be used.")
except Exception as e:
    print(f"Error loading processor_ratings.csv: {str(e)}")

def get_processor_rank(processor_name, gpu_name, rankings_dict):
    """
    Get processor rank from the pre-loaded rankings dictionary.
    Returns rank (1-208) or a default value if not found.
    """
    if pd.isna(processor_name) or not isinstance(processor_name, str):
        print(f"Invalid processor name: {processor_name} - Using default rank: {len(rankings_dict) + 1}")
        return len(rankings_dict) + 1

    # Clean the processor name for lookup
    cleaned_name = processor_name.lower().strip()
    
    # First try exact match
    if cleaned_name in rankings_dict:
        processor_info = rankings_dict[cleaned_name]
        print(f"Found exact match: {processor_name} - Rank: {processor_info['rank']}")
        return processor_info['rank']
    
    # If no exact match, try partial matching
    for key, info in rankings_dict.items():
        # For "Other" company processors, only match processor name
        if info['company'] == 'other':
            if info['processor_name'] in cleaned_name or cleaned_name in info['processor_name']:
                print(f"Found partial match (Other): {processor_name} -> {key} - Rank: {info['rank']}")
                return info['rank']
        else:
            # For other companies, match full name
            if cleaned_name in key or key in cleaned_name:
                print(f"Found partial match: {processor_name} -> {key} - Rank: {info['rank']}")
                return info['rank']
    
    # If no match found, try to find similar processors
    similar_processors = []
    
    # First check if it's an "Other" company processor
    other_series_patterns = ['a', 'apple', 'helio', 'dimensity', 'tensor', 'exynos', 'kirin', 'unisoc', 'tiger', 't']
    for pattern in other_series_patterns:
        if pattern in cleaned_name:
            # Find all processors in the "Other" category with this pattern
            for key, info in rankings_dict.items():
                if info['company'] == 'other' and pattern in info['processor_name']:
                    similar_processors.append(info['rank'])
            if similar_processors:
                median_rank = int(np.median(similar_processors))
                print(f"Found similar 'Other' processor: {processor_name} - Using median rank: {median_rank}")
                return median_rank
    
    # If not an "Other" processor, try regular company matching
    company = None
    series = None
    
    # Try to identify company
    for comp in ['mediatek', 'qualcomm', 'apple', 'samsung', 'google', 'intel', 'amd']:
        if comp in cleaned_name:
            company = comp
            break
    
    # Try to identify series
    series_patterns = {
        'mediatek': ['helio', 'dimensity', 'g', 'p'],
        'qualcomm': ['snapdragon', 'gen', 's'],
        'apple': ['a', 'm'],
        'samsung': ['exynos'],
        'google': ['tensor'],
        'intel': ['core', 'celeron', 'pentium'],
        'amd': ['ryzen']
    }
    
    if company and company in series_patterns:
        for pattern in series_patterns[company]:
            if pattern in cleaned_name:
                series = pattern
                break
    
    # Find similar processors based on company and series
    if company and series:
        for key, info in rankings_dict.items():
            if company in key.lower() and series in key.lower():
                similar_processors.append(info['rank'])
    
    # If we found similar processors, use their median rank
    if similar_processors:
        median_rank = int(np.median(similar_processors))
        print(f"Processor not found: {processor_name} - Using median rank of similar processors: {median_rank}")
        return median_rank
    
    # If no similar processors found, use default rank
    print(f"Processor not found and no similar processors: {processor_name} - Using default rank: {len(rankings_dict) + 1}")
    return len(rankings_dict) + 1

def calculate_performance_score(row, rankings_dict):
    """
    Calculate performance score based on processor ranking and hardware specifications.
    Returns a score between 0-100.
    """
    # Get processor ranking using the loaded dictionary
    processor_name = row.get('chipset')
    gpu_name = row.get('gpu')
    processor_rank = get_processor_rank(processor_name, gpu_name, rankings_dict)

    # Define the maximum possible rank based on your CSV + 1 for not found
    max_possible_rank = len(rankings_dict) + 1

    # Convert processor rank to score (higher rank = lower score)
    if processor_rank > len(rankings_dict):
        processor_score = 0
    else:
        processor_score = 100 - ((processor_rank - 1) / (len(rankings_dict)) * 100)
        processor_score = max(0, processor_score)

    # Rest of the scoring logic remains the same...
    # CPU score
    cpu_score = 0
    cpu_str = row.get('cpu')
    if pd.notna(cpu_str) and isinstance(cpu_str, str):
        cpu_str = cpu_str.lower()
        if 'octa' in cpu_str:
            cpu_score = 80
        elif 'hexa' in cpu_str:
            cpu_score = 60
        elif 'quad' in cpu_str:
            cpu_score = 40
        elif 'dual' in cpu_str:
            cpu_score = 20
        else:
            numbers = re.findall(r'\d+', cpu_str)
            if numbers:
                cpu_score = min(int(numbers[0]) * 10, 100)

    # GPU score
    gpu_score = 50
    if pd.notna(gpu_name) and isinstance(gpu_name, str):
        if 'Adreno' in gpu_name:
            gpu_score = 80
        elif 'Mali' in gpu_name:
            gpu_score = 70
        elif 'PowerVR' in gpu_name:
            gpu_score = 60

    # RAM score
    ram_score = 0
    ram_gb = row.get('ram_gb')
    ram_type = row.get('ram_type')
    if pd.notna(ram_gb):
        ram_score = min(ram_gb * 10, 100)
        if pd.notna(ram_type) and isinstance(ram_type, str):
            if 'LPDDR5' in ram_type:
                ram_score *= 1.2
            elif 'LPDDR4X' in ram_type:
                ram_score *= 1.1

    # Storage score
    storage_score = 0
    storage_gb = row.get('storage_gb')
    storage_type = row.get('storage_type')
    if pd.notna(storage_gb):
        storage_score = min(storage_gb * 0.5, 100)
        if pd.notna(storage_type) and isinstance(storage_type, str):
            if 'UFS 3.1' in storage_type:
                storage_score *= 1.3
            elif 'UFS 3.0' in storage_type:
                storage_score *= 1.2
            elif 'UFS 2.2' in storage_type:
                storage_score *= 1.1

    # Calculate final score with weights
    final_score = (
        processor_score * 0.4 +
        cpu_score * 0.2 +
        gpu_score * 0.2 +
        ram_score * 0.1 +
        storage_score * 0.1
    )

    return round(final_score, 2)

# Apply the function to the DataFrame
if processor_rankings:
    df['performance_score'] = df.apply(lambda row: calculate_performance_score(row, processor_rankings), axis=1)
else:
    print("Skipping performance score calculation due to missing processor rankings.")
    df['performance_score'] = np.nan

Successfully loaded 210 processor rankings from processor_ratings.csv
Found similar 'Other' processor: Qualcomm SM8750-AC Snapdragon 8 Elite - Using median rank: 52
Found similar 'Other' processor: Apple A19 Pro - Using median rank: 52
Found similar 'Other' processor: Apple A19 - Using median rank: 52
Found similar 'Other' processor: Qualcomm SM8635 Snapdragon 8s Gen 3 - Using median rank: 52
Found exact match: Mediatek Dimensity 9400 Plus - Rank: 1
Found similar 'Other' processor: Qualcomm SM8735 Snapdragon 8s Gen 4 - Using median rank: 52
Found partial match (Other): Mediatek Helio G81 -> helio g81 - Rank: 153
Found exact match: Qualcomm Snapdragon 7 Gen 4 - Rank: 39
Found similar 'Other' processor: Unisoc T7250 - Using median rank: 113
Found partial match (Other): MediaTek Helio G100 Ultimate -> helio g100 - Rank: 111
Found similar 'Other' processor: Mediatek Helio G200 - Using median rank: 52
Found exact match: Unisoc SC9863A - Rank: 204
Found partial match (Other): Mediatek Helio 

In [22]:
df[['name', 'chipset', 'gpu','ram', 'performance_score']].head()

,name,chipset,gpu,ram,performance_score
0,Samsung Galaxy Z Fold7 (512GB),Qualcomm SM8750-AC Snapdragon 8 Elite,Adreno 830 (1200 MHz),12 GB,82.29
1,Apple iPhone 17 Pro Max,Apple A19 Pro,Apple GPU (6-core graphics),12 GB,72.29
2,Apple iPhone 17 Air,Apple A19,Apple GPU,8 GB,66.69
3,OnePlus Nord 5,Qualcomm SM8635 Snapdragon 8s Gen 3,Adreno 735,12 GB,87.29
4,Xiaomi Redmi K80 Ultra,Mediatek Dimensity 9400 Plus,Immortalis-G925,24 GB,76.00


Security-based Features:
Connectivity-based Features:
Brand-based Features:
Release-based Features:
Composite Features:

##### Security-based Features

In [23]:
def calculate_security_score(row):
    """
    Calculate security score based on available security features.
    Returns a score between 0-100.
    """
    security_score = 0
    max_score = 100
    weights = {
        'biometric': 0.4,      # 40% weight for biometric security
        'os_security': 0.3,    # 30% weight for OS security
        'build_security': 0.3  # 30% weight for physical security
    }
    
    # 1. Biometric Security Score (40%)
    biometric_score = 0
    
    # Fingerprint sensor
    if pd.notna(row.get('fingerprint_sensor')) and row['fingerprint_sensor']:
        biometric_score += 20
        # Additional points for fingerprint sensor type
        if pd.notna(row.get('finger_sensor_type')):
            sensor_type = str(row['finger_sensor_type']).lower()
            if 'ultrasonic' in sensor_type:
                biometric_score += 10  # Ultrasonic is most secure
            elif 'optical' in sensor_type:
                biometric_score += 8   # Optical is second best
            elif 'in display' in sensor_type:
                biometric_score += 7   # In-display is third best
            elif 'side' in sensor_type:
                biometric_score += 5   # Side-mounted is basic
    
    # Face unlock
    if pd.notna(row.get('face_unlock')) and row['face_unlock']:
        biometric_score += 10
    
    # 2. OS Security Score (30%)
    os_score = 0
    
    # OS Type and Version
    if pd.notna(row.get('operating_system')):
        os = str(row['operating_system']).lower()
        
        # Modern OS scoring
        if 'ios' in os:
            os_score += 15  # iOS has best security
        elif 'harmonyos' in os:
            os_score += 12  # HarmonyOS is modern and secure
        elif 'android' in os:
            os_score += 10  # Android is secure but less than iOS
            # Try to extract version for Android
            try:
                version = float(''.join(filter(str.isdigit, os)))
                os_score += min((version / 14) * 5, 5)  # Additional points for newer versions
            except:
                pass
        elif 'flyme' in os or 'flyme aios' in os:
            os_score += 8   # Flyme is a custom Android with security features
        elif 'tizen' in os:
            os_score += 7   # Tizen is relatively secure
        else:
            os_score += 5   # Basic score for other modern OS
    
    # 3. Physical Security Score (30%)
    physical_score = 0
    
    # IP Rating scoring
    if pd.notna(row.get('ip_rating')):
        ip_rating = str(row['ip_rating']).lower()
        
        # Highest protection
        if any(x in ip_rating for x in ['ip68/ip69', 'ip69', 'ip68/ip69k', 'ip66/ip68/ip69']):
            physical_score += 15
        # High protection
        elif any(x in ip_rating for x in ['ip68', 'ip66/ip68', 'ip65/ip68']):
            physical_score += 12
        # Medium-high protection
        elif any(x in ip_rating for x in ['ip67', 'ip66', 'ip65']):
            physical_score += 10
        # Medium protection
        elif any(x in ip_rating for x in ['ip64', 'ip54', 'ip53']):
            physical_score += 8
        # Basic protection
        elif any(x in ip_rating for x in ['ip52', 'ip50', 'ip48']):
            physical_score += 5
    
    # Ruggedness scoring
    if pd.notna(row.get('ruggedness')):
        ruggedness = str(row['ruggedness']).lower()
        if 'dust proof' in ruggedness or 'dustproof' in ruggedness:
            physical_score += 15
        elif 'dust resistant' in ruggedness:
            physical_score += 10
        elif 'ip53' in ruggedness:  # Some entries have IP53 in ruggedness
            physical_score += 8
    
    # Calculate final weighted score
    final_score = (
        biometric_score * weights['biometric'] +
        os_score * weights['os_security'] +
        physical_score * weights['build_security']
    )
    
    return round(final_score, 2)

# Apply the function to the DataFrame
df['security_score'] = df.apply(calculate_security_score, axis=1)

In [24]:
finger_types = df['finger_sensor_type'].unique().tolist()
os = df['operating_system'].unique().tolist()
ip = df['ip_rating'].unique().tolist()
rug = df['ruggedness'].unique().tolist()

print('finger sensor types:',finger_types)
print('Operating systems:',os)
print('ip ratings:',ip)
print('ruggedness',rug)


finger sensor types: [nan, 'Optical', 'Ultrasonic', 'Side-Mounted', 'Side-mounted', 'In Display']
Operating systems: ['Android', 'iOS', 'HarmonyOS', 'Feature phones', 'Feature Phone', 'Flyme', 'FeaturePhone', 'SkyUI', 'Flyme AIOS', nan, 'RTOS OS', 'S30+', 'KaiOS', 'KAI OS', 'Android 10', 'Android v10 (Q)', 'Android v10', 'Java', 'Java Me', 'Feature phone', 'Microsoft Windows', 'Microsoft Windows Phone', 'Windows Phone', 'Tizen', 'Firefox OS', 'Windows', 'Nokia Asha', 'Symbian', 'Symbian Belle', 'TouchWiz UI', 'Symbian^3', 'Maemo', 'Microsoft Windows Mobile', 'Symbian OS', 'Symbian OS v9.2']
ip ratings: ['IP48', 'IP68', 'IP65', 'IP68/IP69', 'IP64', 'IP69', nan, 'IP58/IP59+', 'IP54', 'IP68/IP69K', 'IP65/IP68', 'IP50', 'IP67', 'IP66/IP68', 'IP53', 'IP66', 'IP66/IP68/IP69', 'IPX8', 'IP65M', 'IPX8/IPX9', 'IP52', 'IPX2', 'IP65, IP68', 'IP68, IP69', 'IP68, IP69K', 'IP69/IP68', 'IPX4', 'IPX4, IP5X', 'IPX5, IPX8, IP6X', 'IP5X, IPX4', 'IPX4/IP5X', 'IPX4, IPX5', 'IP86', 'IPX7', 'IP58', 'IP57']
ru

In [25]:
df[['name','fingerprint_sensor','finger_sensor_type', 'operating_system', 'ip_rating' ,'security_score']].head()

,name,fingerprint_sensor,finger_sensor_type,operating_system,ip_rating,security_score
0,Samsung Galaxy Z Fold7 (512GB),Yes,NaN,Android,IP48,21.0
1,Apple iPhone 17 Pro Max,NaN,NaN,iOS,IP68,16.6
2,Apple iPhone 17 Air,NaN,NaN,iOS,IP68,16.6
3,OnePlus Nord 5,Yes,Optical,Android,IP65,25.7
4,Xiaomi Redmi K80 Ultra,Yes,Ultrasonic,Android,IP68/IP69,28.0


##### Connectivity-based Features

In [26]:
def calculate_connectivity_score(row):
    """
    Calculate connectivity score based on various connectivity features.
    Returns a score between 0-100.
    """
    connectivity_score = 0
    max_score = 100
    weights = {
        'network': 0.3,      # 30% weight for network connectivity
        'wireless': 0.3,     # 30% weight for wireless connectivity
        'ports': 0.2,        # 20% weight for physical ports
        'sim': 0.2          # 20% weight for SIM capabilities
    }
    
    # 1. Network Connectivity Score (30%)
    network_score = 0
    
    # Network type and speed
    if pd.notna(row.get('speed')):
        speed = str(row['speed']).lower()
        
        # 5G scoring
        if '5g' in speed:
            network_score += 15
            # Additional points for 5G speed
            if '7.5' in speed or '7.5 gbps' in speed:
                network_score += 5
            elif '5+' in speed or '5 gbps' in speed:
                network_score += 4
            elif '4.5' in speed or '4.4' in speed or '4 gbps' in speed:
                network_score += 3
            elif '3.3' in speed or '3.6' in speed or '3.7' in speed:
                network_score += 2
            elif '2.3' in speed or '2.4' in speed or '2+' in speed:
                network_score += 1
        
        # LTE scoring
        if 'lte' in speed:
            network_score += 10
            # Additional points for LTE-A and carrier aggregation
            if 'lte-a' in speed:
                network_score += 5
            if 'ca' in speed:
                ca_count = 0
                if '7ca' in speed:
                    ca_count = 7
                elif '6ca' in speed:
                    ca_count = 6
                elif '5ca' in speed:
                    ca_count = 5
                elif '4ca' in speed:
                    ca_count = 4
                elif '3ca' in speed:
                    ca_count = 3
                elif '2ca' in speed:
                    ca_count = 2
                network_score += min(ca_count * 2, 10)
    
    # 2. Wireless Connectivity Score (30%)
    wireless_score = 0
    
    # WLAN scoring
    if pd.notna(row.get('wlan')):
        wlan = str(row['wlan']).lower()
        
        # WiFi version scoring
        if 'wifi 8' in wlan:
            wireless_score += 15
        elif 'wifi 7' in wlan:
            wireless_score += 14
        elif 'wifi 6e' in wlan:
            wireless_score += 12
        elif 'wifi 6' in wlan:
            wireless_score += 10
        elif 'wifi 5' in wlan:
            wireless_score += 8
        elif 'wifi 4' in wlan:
            wireless_score += 5
        elif 'wifi 3' in wlan:
            wireless_score += 3
        elif 'wifi 2' in wlan:
            wireless_score += 2
        
        # Additional features
        if 'mimo' in wlan:
            wireless_score += 5
        if '6ghz' in wlan:
            wireless_score += 3
        if '5ghz' in wlan:
            wireless_score += 2
        if 'dual-band' in wlan:
            wireless_score += 2
    
    # Bluetooth scoring
    if pd.notna(row.get('bluetooth')):
        bluetooth = str(row['bluetooth']).lower()
        
        # Version scoring
        if 'v6.0' in bluetooth or '6.0' in bluetooth:
            wireless_score += 10
        elif 'v5.4' in bluetooth or '5.4' in bluetooth:
            wireless_score += 9
        elif 'v5.3' in bluetooth or '5.3' in bluetooth:
            wireless_score += 8
        elif 'v5.2' in bluetooth or '5.2' in bluetooth:
            wireless_score += 7
        elif 'v5.1' in bluetooth or '5.1' in bluetooth:
            wireless_score += 6
        elif 'v5.0' in bluetooth or '5.0' in bluetooth:
            wireless_score += 5
        elif 'v4.2' in bluetooth or '4.2' in bluetooth:
            wireless_score += 4
        elif 'v4.1' in bluetooth or '4.1' in bluetooth:
            wireless_score += 3
        elif 'v4.0' in bluetooth or '4.0' in bluetooth:
            wireless_score += 2
        
        # Additional features
        if 'aptx' in bluetooth:
            wireless_score += 2
        if 'le' in bluetooth:
            wireless_score += 1
    
    # 3. Physical Ports Score (20%)
    ports_score = 0
    
    # USB
    if pd.notna(row.get('usb')):
        usb = str(row['usb']).lower()
        if 'usb-c' in usb or 'usb c' in usb:
            ports_score += 10
        elif 'usb' in usb:
            ports_score += 5
    
    # USB OTG
    if pd.notna(row.get('usb_otg')) and row['usb_otg']:
        ports_score += 5
    
    # 4. SIM Capabilities Score (20%)
    sim_score = 0
    
    # SIM Slot
    if pd.notna(row.get('sim_slot')):
        sim = str(row['sim_slot']).lower()
        if 'dual' in sim:
            sim_score += 10
        elif 'single' in sim:
            sim_score += 5
    
    # VoLTE
    if pd.notna(row.get('volte')) and row['volte']:
        sim_score += 10
    
    # Calculate final weighted score
    final_score = (
        network_score * weights['network'] +
        wireless_score * weights['wireless'] +
        ports_score * weights['ports'] +
        sim_score * weights['sim']
    )
    
    return round(final_score, 2)

# Apply the function to the DataFrame
df['connectivity_score'] = df.apply(calculate_connectivity_score, axis=1)

In [27]:
wifi = df['wlan'].unique().tolist()
speed = df['speed'].unique().tolist()
bluetooth = df['bluetooth'].unique().tolist()
usb = df['usb'].unique().tolist()

print('wlan types:',wifi)
print('speed types:',speed)
print('bluetooth types:',bluetooth)
print('usb',usb)

wlan types: ['Wi-Fi 7 (802.11 a/b/g/n/ac/be/ax) 5GHz 6GHz, MIMO', 'Wi-Fi 6 (802.11 a/b/g/n/ac/ax) 5GHz, MIMO', 'Wi-Fi 5 (802.11 a/b/g/n/ac) 5GHz', 'Wi-Fi 4 (802.11 b/g/n)', 'Wi-Fi 7 (802.11 a/b/g/n/ac/be/ax) 5GHz 6GHz', nan, 'Wi-Fi 6E (802.11 a/b/g/n/ac/ax) 5GHz 6GHz, MIMO', 'Wi-Fi 6 (802.11 a/b/g/n/ac/ax) 5GHz', 'Wi-Fi 5 (802.11 a/b/g/n/ac)', 'Wi-Fi 6E (802.11 a/b/g/n/ac/ax) 5GHz 6GHz', 'Wi-Fi 4 (802.11 a/b/g/n)', 'Wi-Fi 5 (802.11 a/b/g/n/ac) 5GHz, MIMO', 'Wi-Fi 6E (802.11 a/b/g/n/ax) 5GHz 6GHz', 'Wi-Fi 5 (802.11 b/g/n/ac)', 'Wi-Fi 6 (802.11 a/b/g/n/ac/ax) 5GHz 6GHz, MIMO', 'Wi-Fi 4 (802.11 a/b/g/n/ac)', 'Wi-Fi 5 (802.11 b/g/n/ac) 5GHz', 'Wi-Fi 6 (802.11 b/g/n/ac) 5GHz', 'Wi-Fi 4 (802.11 b/g/n), MIMO', 'Wi-Fi 802.11 a/b/g/n/ac, dual-band', 'Wi-Fi 5 (802.11 a/b/e/g/n/ac) 5GHz, MIMO', 'Wi-Fi 5 (802.11 b/g/n)', 'Wi-Fi 6E (802.11 a/b/g/n/ac/ax) 6GHz, MIMO', 'Wi-Fi 5 (802.11 a/b/g/n/ac/6e) 5GHz', 'Wi-Fi 4 (802.11 b/g/n) 5GHz', 'Wi-Fi 5 (802.11 a/b/n/ac)', 'Wi-Fi 7 (802.11 a/b/g/n/x/ac/be/a

In [28]:
df[['name', 'connectivity_score']].head()

,name,connectivity_score
0,Samsung Galaxy Z Fold7 (512GB),20.2
1,Apple iPhone 17 Pro Max,20.2
2,Apple iPhone 17 Air,18.9
3,OnePlus Nord 5,19.3
4,Xiaomi Redmi K80 Ultra,20.2


##### Brand-based Features

In [29]:
brand = df['brand'].unique().tolist()

print('brand list', brand)

brand list ['Samsung', 'Apple', 'OnePlus', 'Xiaomi', 'Nothing', 'Tecno', 'Realme', 'Infinix', 'ZTE', 'Honor', 'Proton', 'Itel', 'Vivo', 'Oppo', 'Lava', 'iQOO', 'Umidigi', 'Huawei', 'Motorola', 'Oscal', 'Sharp', 'Walton', 'Sony', 'Symphony', 'Acer', 'Doogee', 'Alcatel', 'Meizu', 'TCL', 'Benco', 'Coolpad', 'Ulefone', 'Google', 'Helio', 'HTC', 'HMD', 'Nokia', 'Energizer', 'Asus', 'Thuraya', 'Sonim', 'Philips', 'Blackview', 'Oukitel', 'Cubot', 'DOOGEE', 'Nio', 'Geo', 'Hallo', 'Wiko', 'PROTON', 'FreeYond', '5star', 'Maximus', 'Leica', 'Lenovo', 'Bengal', 'LG', 'Leitz', nan, 'maximus', 'TECNO', 'vivo', 'UMIDIGI', 'We', 'WE', 'Panasonic', 'Micromax', 'Allview', 'WE X2', 'Cat', 'XTRA', 'Mycell', 'BlackBerry', 'Okapia', 'Oneplus', 'Kingstar', 'LAVA', 'Microsoft', 'Gionee', 'OnePlus 2', 'Celkon', 'Maxis']


In [31]:
def is_popular_brand(brand):
    """
    Determine if a brand is considered popular based on market presence and recognition.
    Returns True for popular brands, False otherwise.
    """
    if pd.isna(brand):
        return False
        
    popular_brands = {
        'Samsung', 'Apple', 'Xiaomi', 'Oppo', 'Vivo', 'OnePlus', 'Realme', 
        'Huawei', 'Honor', 'Motorola', 'Google', 'Nokia', 'Asus', 'Sony',
        'LG', 'Lenovo', 'HTC', 'BlackBerry', 'Microsoft', 'Panasonic'
    }
    
    return str(brand).strip() in popular_brands

# Create derived column for popular brands
df['is_popular_brand'] = df['brand'].apply(is_popular_brand) 

df[['name', 'brand', 'is_popular_brand']].head()

,name,brand,is_popular_brand
0,Samsung Galaxy Z Fold7 (512GB),Samsung,True
1,Apple iPhone 17 Pro Max,Apple,True
2,Apple iPhone 17 Air,Apple,True
3,OnePlus Nord 5,OnePlus,True
4,Xiaomi Redmi K80 Ultra,Xiaomi,True


##### Release-based features

In [32]:
from datetime import datetime, timedelta

def clean_release_date(date_str):
    """
    Clean and standardize release date format.
    Returns datetime object or None if invalid.
    """
    if pd.isna(date_str):
        return None
        
    date_str = str(date_str).strip()
    
    # Handle "Not announced yet" case
    if date_str.lower() in ['not announced yet', 'not announced', 'tba', 'tbd']:
        return None
        
    try:
        # Handle "Exp. DD Month YYYY" format
        if date_str.startswith('Exp.'):
            date_str = date_str.replace('Exp.', '').strip()
            date_formats = ['%d %B %Y', '%d %b %Y']
        # Handle "DD-Month-YY" format
        elif '-' in date_str:
            date_formats = ['%d-%b-%y']
        else:
            date_formats = ['%d %B %Y', '%d %b %Y', '%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y']
        
        for fmt in date_formats:
            try:
                return datetime.strptime(date_str, fmt)
            except ValueError:
                continue
                
        return None
    except:
        return None

def is_new_release(release_date):
    """
    Check if the device was released within the last 6 months.
    Returns boolean.
    """
    if pd.isna(release_date):
        return False
        
    six_months_ago = datetime.now() - timedelta(days=180)
    return release_date >= six_months_ago

def calculate_age_in_months(release_date):
    """
    Calculate the age of the device in months.
    Returns float or None if invalid.
    """
    if pd.isna(release_date):
        return None
        
    try:
        age_days = (datetime.now() - release_date).days
        return round(age_days / 30.44, 2)  # Average days in a month
    except:
        return None

def is_upcoming(status):
    """
    Check if the device is upcoming or rumored.
    Returns boolean.
    """
    if pd.isna(status):
        return False
        
    upcoming_statuses = ['upcoming', 'rumored', 'expected', 'announced', 'not announced yet', 'tba', 'tbd']
    return str(status).lower().strip() in upcoming_statuses

# Create derived columns for release features
df['release_date_clean'] = df['release_date'].apply(clean_release_date)
df['is_new_release'] = df['release_date_clean'].apply(is_new_release)
df['age_in_months'] = df['release_date_clean'].apply(calculate_age_in_months)
df['is_upcoming'] = df['status'].apply(is_upcoming)

In [33]:
df[['name', 'release_date','release_date_clean', 'is_new_release', 'age_in_months', 'age_in_months']].head()

,name,release_date,release_date_clean,is_new_release,age_in_months,age_in_months
0,Samsung Galaxy Z Fold7 (512GB),9-Jul-25,2025-07-09,True,0.33,0.33
1,Apple iPhone 17 Pro Max,Not announced yet,NaT,False,NaN,NaN
2,Apple iPhone 17 Air,Not announced yet,NaT,False,NaN,NaN
3,OnePlus Nord 5,9-Jul-25,2025-07-09,True,0.33,0.33
4,Xiaomi Redmi K80 Ultra,Not announced yet,NaT,False,NaN,NaN


##### Composite Features

In [34]:
# Calculate overall device score (weighted average of all scores)
df['overall_device_score'] = (
    df['performance_score'] * 0.3 +
    df['connectivity_score'] * 0.2 +
    df['camera_score'] * 0.3 +
    df['battery_score'] * 0.2
).round(2)

df[['name', 'performance_score', 'connectivity_score', 'camera_score', 'battery_score', 'overall_device_score']].head()

,name,performance_score,connectivity_score,camera_score,battery_score,overall_device_score
0,Samsung Galaxy Z Fold7 (512GB),82.29,20.2,69.69,44.72,58.58
1,Apple iPhone 17 Pro Max,72.29,20.2,38.25,45.97,46.40
2,Apple iPhone 17 Air,66.69,18.9,22.62,34.17,37.41
3,OnePlus Nord 5,87.29,19.3,45.94,60.56,55.94
4,Xiaomi Redmi K80 Ultra,76.00,20.2,36.88,66.67,51.24


In [35]:
print(df.columns.to_list())

['name', 'brand', 'model', 'price', 'url', 'img_url', 'display_type', 'screen_size_inches', 'display_resolution', 'pixel_density_ppi', 'refresh_rate_hz', 'screen_protection', 'display_brightness', 'aspect_ratio', 'hdr_support', 'chipset', 'cpu', 'gpu', 'ram', 'ram_type', 'internal_storage', 'storage_type', 'camera_setup', 'primary_camera_resolution', 'selfie_camera_resolution', 'main_camera', 'front_camera', 'primary_camera_video_recording', 'selfie_camera_video_recording', 'primary_camera_ois', 'primary_camera_aperture', 'image_resolution', 'selfie_camera_aperture', 'camera_features', 'autofocus', 'flash', 'settings', 'zoom', 'shooting_modes', 'video_fps', 'battery_type', 'capacity', 'quick_charging', 'wireless_charging', 'reverse_charging', 'build', 'weight', 'thickness', 'colors', 'waterproof', 'ip_rating', 'ruggedness', 'network', 'speed', 'sim_slot', 'volte', 'bluetooth', 'wlan', 'gps', 'nfc', 'usb', 'usb_otg', 'fingerprint_sensor', 'finger_sensor_type', 'finger_sensor_position', 

In [36]:
# Save the DataFrame to CSV
df.to_csv('mobile_data.csv', index=False)
